In [1]:
import numpy as np
import skimage
import cv2
import os
import matplotlib.pyplot as plt
from PIL import Image
from skimage import feature
from pathlib import Path
# from fastai.vision.all import *

In [2]:
class LocalBinaryPattern:
    def __init__(self, num_points, radius):
        self.num_points = num_points 
        self.radius = radius
        
    def describe(self, image, eps=1e-7):
        lbp = feature.local_binary_pattern(image, self.num_points, self.radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(),
        bins=np.arange(0, self.num_points + 3),
        range=(0, self.num_points + 2))
    # normalize the histogram
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)
        return hist

In [67]:
image  = Image.open('data/spoof_crop/real/1_0_0.jpg')
# image = Image.open('data/spoof_crop/fake/3_3_0_0.jpg')

In [68]:
arr = np.array(image)

In [69]:
gray = cv2.cvtColor(arr, cv2.COLOR_RGB2GRAY)

In [70]:
lbp = LocalBinaryPattern(16, 3)

In [3]:
path = Path('data/spoof_crop')

In [4]:
path_real = path/'real'
path_fake = path/'fake'

In [6]:
def extract_feature(image_path, lbp):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    hist = lbp.describe(gray)
    return hist

In [7]:
data = []
label = []

In [9]:
lbp = LocalBinaryPattern(16, 3)

In [12]:
for image in os.listdir(path_real):
    image_path = str(path_real/image)
    hist = extract_feature(image_path, lbp)
    
    data.append(hist)
    label.append(1)

In [13]:
for image in os.listdir(path_fake):
    image_path = str(path_fake/image)
    hist = extract_feature(image_path, lbp)
    
    data.append(hist)
    label.append(0)

In [16]:
data = np.array(data)
label = np.array(label)

<function numpy.save(file, arr, allow_pickle=True, fix_imports=True)>

In [22]:
np.save('data.npy', np.concatenate([data, label.reshape(-1, 1)], axis=-1))

In [18]:
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [4]:
data = np.load('data.npy')

In [5]:
X, y = data[:, :-1], data[:, -1]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

In [33]:
model = RandomForestClassifier(1000, criterion='entropy', min_samples_leaf=8, max_features=0.6,
                             n_jobs=-1, oob_score=True)
model.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', max_features=0.6,
                       min_samples_leaf=8, n_estimators=1000, n_jobs=-1,
                       oob_score=True)

In [41]:
model = SVC(kernel='poly')
model.fit(X_train, y_train)

SVC(kernel='poly')

In [42]:
model.score(X_test, y_test), model.score(X_train, y_train)

(0.8043831168831169, 0.7861494719740049)